## Char 8 
## learning signal and ignoring noise: introduction to regularization and batching

#### In this chapter
* Overfitting
* Dropout
* Batch gradient descent

<table align="left">
  <td>
    <a><img src="./dropout.jpg", width=600, heigth=600></a>
  </td>
</table>

신경망의 학습능력은 뉴런 수와 신경망 차원의 영향을 받는데, 뉴런 수가 많을수록, 신경망 차원이 높을수록 그 신경망의 학습능력이 강할수록 의합되는 overfitting문제가 발생할 수 있다.\
이 장에서 우리는 정칙화된 기초 지식을 연구할 것이다.모형의 복잡도를 낮추어, 네트워크에서 overfitting문제를 단속하다.우선 강력한 기능의 신경망(레루의 숨은 층을 가진 3층 네트워크)에서 시작하고 가장 도전적인 임무(MNIST 숫자 분류)에 대비하는 것이다.

### Three-layer network on MNIST

In [1]:
from tensorflow.keras.datasets import mnist

import numpy as np
import sys

(x_train, y_train), (x_test, y_test) = mnist.load_data() #导入数据,划分数据集

images, labels = (x_train[0:1000].reshape(1000,28*28)/255.0, y_train[0:1000])

one_hot_labels = np.zeros((len(labels),10)) # 独热 只有一位是有效的[0,0,0,1,0,0,0]

for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test), 28*28)/255# 并把像素压缩到0，1之间
test_labels = np.zeros((len(y_test),10))

for i,l in enumerate(y_test):
    test_labels[i][l] = 1
np.random.seed(1) 
#设置激活函数active function
relu = lambda x:(x>=0) * x # return x if x > 0; returns 0 otherwise
relu2deriv = lambda x: x>=0 # return 1 for input > 0;returns 0 otherwise
alpha, iterations,hidden_size, pixels_per_image, num_labels = (0.005,350, 40, 784, 10)
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

print(weights_0_1.shape)

for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        layer_2 = np.dot(layer_1, weights_1_2)
        error += np.sum((labels[i:i+1] - layer_2) ** 2)
        correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))
        layer_2_delta = (labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)*relu2deriv(layer_1)
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
        
    sys.stdout.write("\r"+ "I:" + str(j) + " Error:" + str(error/float(len(images)))[0:5] + " Correct:" + str(correct_cnt/float(len(images))) )

C:\Users\jia97\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jia97\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jia97\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint16 = np.dtype([("quint16", np.uint16, 1)])
C:\Users\jia97\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Pa

(784, 40)
I:349 Error:0.108 Correct:1.099

In [2]:
print(one_hot_labels.shape)

(1000, 10)


다음의 코드를 실행하여, 이 영상들에서 신경망을 실행하고 그것들을 네트워크가 얼마나 분류하는지 평가하게 된다:

In [3]:
if(j % 10 == 0 or j == iterations-1):
    error, correct_cnt = (0.0, 0)
    
    for i in range(len(test_images)):
        layer_0 = test_images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        layer_2 = np.dot(layer_1, weights_1_2)
        
        error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
        correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
        
    # 测试准确性：预测准确率只有70.7%
    sys.stdout.write(" Test-Err:" + str(error/float(len(test_images)))[0:5] + " Test_Acc:" + str(correct_cnt/float(len(test_images))))
    print()

 Test-Err:0.653 Test_Acc:0.7073


여기에는 training과 testing에서 테스트한 신경망 정확성이 함께 프린트돼 있다.(10회 반복)

<table align="left">
  <td>
    <a><img src="./train_test.png", width=600, heigth=400></a>
  </td>
</table>

어떤 이유로 Test정도는 (상위 20번) Iteration에서 먼저 올렸다가 낮춘다 (그러나 훈련의 정확성은 여전히 개선되고 있다)\

* 익숙해 보이지 않는 것을 주면 무작위로 예측한다.
如果给他看起来不熟悉的东西，它将随机预测

그러나, 테스트 중인 이미지 dataset과 훈련 dataset의 패턴은 약간 다르다.
* 카테고리별 비율: 예를 들어 train dataset에 하는 차 30%, 사람 40%, 나무 30% 가 있다. test dataset는 차 10%, 사람 20%, 나무 70%가 있다.그렇다면 훈련 dataset는 거리 풍경에서 나온 것일 수 있다.test dataset의 데이터는 공원 장면에서 나온 것으로 보인다.양자의 각 종류별 비율이 다르기 때문에 데이터 분포는 다르다.
* 이미지 특징: 예를 들어 훈련dataset는 우리가 인터넷에서 수집하는 매우 선명한 고양이와 개다이미지, test dataset는 우리가 직접 찍은 흐릿한 고양이와 개의 이미지.혹은 훈련dataset가 흰색 개와 노란색 고양이, test dataset가 노란색 개와 흰색 고양이의 이미지라면 데이터 분포도 다르다고 볼 수 있다.

Overfitting의 신경망:Dataset에서 나는 noise을 습득한 것이지 실제 신호만으로 얻어지는 것은 아니었다.

#### 무엇이 신경망 과의합성을 초래하다?

과체결 현상은 일반적으로 공부가 지나치게 정확하기 때문이다.마치 사람 얼굴을 기계로 배워 100명의 얼굴을 훈련시키는 것과 비슷하지만, 당신이 정확히 공부한 덕분에 이 샘플을 제외한 다른 사람 얼굴 신경망이 사람 얼굴이 아니라고 한다.실제 사람 얼굴의 기본 특징만 배우면 되는데, 사람의 피부 세밀함, 눈 크기 등 지나치게 섬세한 특징이 아니라 다른 사진 속 사람의 얼굴을 인식할 수 있다.

### The simplest regularization :Early stopping

1.한 가지 방법으로 가능한 한 빨리 정지한다.\
2.언제 멈출지 어떻게 알아.\
훈련 dataset에 없는 데이터에 액세스하는 모델이다.보통 두 번째 테스트 dataset을 검증집이라고 한다.test dataset을 언제 멈추는지 아는 데 사용하면 계측기에 과도하게 적응할 수 있다.그래서 보통 검증집을 쓴다.

### Industry standard regularization: Dropout
The method: Randomly turn off neurons(set them to 0)during training.

네트워크에서 뉴런은 무작위로 0으로 설정한다.이런 정규화 기술은 절대 다수의 네트워크에 적용되는 최첨단 정규화 기술로 널리 인식되고 있다.방법은 간단하고 비싸지 않다. 뒤가 좀 복잡할 수 있다.

### Why dropout works

Dropout은 무작위로 일부 네트워크를 훈련시켜 큰 네트워크를 작은 네트워크처럼 만들었다.작은 네트워크는 과도하게 적응하지 않는다.

신경망이 작을수록 과도하게 작성하는 능력이 줄어든다는 사실이 밝혀졌다.

원인:작은 신경망에는 표현력이 별로 없다.좀 더 미세한 디테일(noise)을 잡지 못하다.여기서는 구역을 정해 관건이 되는 부분을 포착하고 세부 사항은 소홀히 한다.그만큼 크고, 분명하고, 고급스러우며, 큰 공간에서만 포획한다는 것이다.
#### 그렇다면 어떻게 우리는 큰 네트워크의 능력을 얻으면서 동시에 작은 신경 네트워크의 과의합에 저항할 수 있는가?

대형 네트워크에서는 기관폐쇄 신경원점(신경절점)을 따르지만 수만 개의 서브넷에서 이를 무작위로 실행해 전체 네트워크를 합친 표현력은 유지된다.

## Why dropout works: Ensembling works
Dropout is a form of training a bunch of networks and averaging them.\
Dropout是训练一群网络并将其平均化的方式。

예를 들어 100개의 신경망(모든 신경망은 무작위로 초기화)을 훈련시키면 신경망마다 다른 소음을 갖게 된다. 그래서 그들은 실수를 하면 다른 실수를 한다. 평균을 내라고 하면 소음이 상쇄되고 신경망이 다른 뉴런들과 더 강건한 로버스트의 특징을 배울 수 있게 된다.\
이렇게 하면 Dropout이 overfitting을 줄일 수 있을 뿐 아니라 정확도를 높일 수 있다.

## Dropout in code
### Here's how to use dropout in the real world.
在MNIST手写数字分类模型里，在隐藏层中添加dropout，有50%的节点被随机关闭。\
MNIST 핸드헬드 디지털 카테고리 모델에서는 hidden층에 dropout을 추가해 50%의 비트가 기관 폐쇄됐다.

<table align="left">
  <td>
    <a><img src="./mask_added.png", width=500, heigth=800></a>
  </td>
</table>

* 층 1에 2를 곱하면 층 2에서 층 1의 가중치를 부여한다.원인: 층 1의 절반 노드를 닫으면 총합이 절반으로 줄어든다.층 2는 층 1의 민감도를 높여 마치 한 사람이 라디오에 접근할 때 음량이 너무 작아 잘 들리지 않는다.드롭아웃을 사용하지 않으면 볼륨이 정상으로 돌아와 청력(층 2)을 파괴한다.
* 층 1에 0과 1의 무작위 행렬을 곱한 값이다.노드를 0으로 설정하고 계층 1의 노드를 임의로 종료하다.

In [4]:
import numpy as np
import sys

np.random.seed(1)
def relu(x):
    return (x >= 0) * x

def relu2deriv(output):
    return output >= 0

alpha, iterations, hidden_size = (0.005, 300, 100)
pixels_per_image, num_labels = (784, 10)

weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
    error, correct_cnt = (0.0,0)
    
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = np.dot(layer_1,weights_1_2)
        
        error += np.sum((labels[i:i+1] - layer_2) ** 2)
        correct_cnt += int(np.argmax(layer_2) == \
                           np.argmax(labels[i:i+1]))
        
        layer_2_delta = (labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
        layer_1_delta *= dropout_mask
        
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
    
    if(j%10 == 0):
        test_error = 0.0
        test_correct_cnt = 0
        
        for i in range(len(test_images)):
            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = np.dot(layer_1, weights_1_2)
            
            test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
            test_correct_cnt += int(np.argmax(layer_2) == \
                                    np.argmax(test_labels[i:i+1]))
            
        sys.stdout.write("\n" + \
"I:" + str(j) + \
" Test-Err:" + str(test_error/ float(len(test_images)))[0:5] +\
" Test-Acc:" + str(test_correct_cnt/ float(len(test_images)))+\
" Train-Err:" + str(error/ float(len(images)))[0:5] +\
" Train-Acc:" + str(correct_cnt/ float(len(images))))
        print()


I:0 Test-Err:0.641 Test-Acc:0.6333 Train-Err:0.891 Train-Acc:0.413

I:10 Test-Err:0.458 Test-Acc:0.787 Train-Err:0.472 Train-Acc:0.764

I:20 Test-Err:0.415 Test-Acc:0.8133 Train-Err:0.430 Train-Acc:0.809

I:30 Test-Err:0.421 Test-Acc:0.8114 Train-Err:0.415 Train-Acc:0.811

I:40 Test-Err:0.419 Test-Acc:0.8112 Train-Err:0.413 Train-Acc:0.827

I:50 Test-Err:0.409 Test-Acc:0.8133 Train-Err:0.392 Train-Acc:0.836

I:60 Test-Err:0.412 Test-Acc:0.8236 Train-Err:0.402 Train-Acc:0.836

I:70 Test-Err:0.412 Test-Acc:0.8033 Train-Err:0.383 Train-Acc:0.857

I:80 Test-Err:0.410 Test-Acc:0.8054 Train-Err:0.386 Train-Acc:0.854

I:90 Test-Err:0.411 Test-Acc:0.8144 Train-Err:0.376 Train-Acc:0.868

I:100 Test-Err:0.411 Test-Acc:0.7903 Train-Err:0.369 Train-Acc:0.864

I:110 Test-Err:0.411 Test-Acc:0.8003 Train-Err:0.371 Train-Acc:0.868

I:120 Test-Err:0.402 Test-Acc:0.8046 Train-Err:0.353 Train-Acc:0.857

I:130 Test-Err:0.408 Test-Acc:0.8091 Train-Err:0.352 Train-Acc:0.867

I:140 Test-Err:0.405 Test-Acc:0


인터넷의 피크는 82.36%에 달했을 뿐 아니라 과도한 의합도 그리 나쁘지 않았고, 훈련 완료 테스트의 정확도는 81.81%에 달했다.여기에 는 Train-acc훈련 속도를 늦추고, 그전까지 100%를 달성하며 그 자리를 지켰다.

* dropout이 소음임을 알 수 있다.훈련 데이터에 대한 인터넷의 훈련이 더 어렵게 된 것이다.마치 마라톤을 다리에 무거운 짐을 진 채 달리는 것과 같다.훈련하기는 어렵지만 큰 경기에 지면 더 빨리 뛸 수 있다.

## Batch gradient descent
#### 훈련 속도와 수렴 속도를 높이다. 提高训练速度和收敛速度

mini_batched stochastic gradient descent(미니 번수 처리 기울기 하강)를 사용한다.한 번에 100개씩 훈련하는 경우, 100개 예에 대한 가중치를 평균한다.\
这里用的是mini_batched stochastic gradient descent(迷你批次处理梯度下降)。一次训练100个训练例子，对100个例子的权重更新求平均。

In [5]:
import numpy as np
np.random.seed(1)

def relu(x):
    return (x >= 0) * x

def relu2deriv(output):
    return output >= 0

batch_size = 100
alpha, iterations = (0.001, 300)
pixels_per_image, num_labels, hidden_size = (784, 10, 100)

weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size),((i+1)*batch_size))
        
        layer_0 = images[batch_start:batch_end]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = np.dot(layer_1,weights_1_2)
        
        error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2)
        for k in range(batch_size):
            correct_cnt += int(np.argmax(layer_2[k:k+1]) == \
                               np.argmax(labels[batch_start+k:batch_start+k+1]))
            layer_2_delta = (labels[batch_start:batch_end]-layer_2)/batch_size
            layer_1_delta = layer_2_delta.dot(weights_1_2.T)*relu2deriv(layer_1)
            layer_1_delta *= dropout_mask
            
            weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
            weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

    if(j%10 == 0):
        test_error = 0.0
        test_correct_cnt = 0
        
        for i in range(len(test_images)):
            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = np.dot(layer_1, weights_1_2)
        
             
        sys.stdout.write("\n" + \
"I:" + str(j) + \
" Test-Err:" + str(test_error/ float(len(test_images)))[0:5] +\
" Test-Acc:" + str(test_correct_cnt/ float(len(test_images)))+\
" Train-Err:" + str(error/ float(len(images)))[0:5] +\
" Train-Acc:" + str(correct_cnt/ float(len(images))))
        print()


I:0 Test-Err:0.0 Test-Acc:0.0 Train-Err:1.284 Train-Acc:0.165

I:10 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.591 Train-Acc:0.672

I:20 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.532 Train-Acc:0.729

I:30 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.498 Train-Acc:0.754

I:40 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.489 Train-Acc:0.749

I:50 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.468 Train-Acc:0.775

I:60 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.452 Train-Acc:0.799

I:70 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.453 Train-Acc:0.792

I:80 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.457 Train-Acc:0.786

I:90 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.454 Train-Acc:0.799

I:100 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.447 Train-Acc:0.796

I:110 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.426 Train-Acc:0.816

I:120 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.431 Train-Acc:0.813

I:130 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.434 Train-Acc:0.816

I:140 Test-Err:0.0 Test-Acc:0.0 Train-Err:0.437 Train-Acc:0.822

I:150 Test-Err:0.0 Test-Acc:0.0 Tra

표시된 훈련/테스트에서 진 것을 보면 훈련의 정밀도가 이전보다 더 평활한 추세를 보이고 있다.\
 따라서 훈련 중 평균적인 update은 학습 과정을 원활하게 할 수 있다고 말했다.

이곳 alpha가 20배는 되지만 속도는 여전히 빠르다는 것을 알 수 있다.
* np.dot 실행은 1회가 벡터점적 100개이기 때문에 CPU가 작동해종자는 훨씬 빨라야 한다.

## Summary
이 장에서는 두 가지 가장 널리 사용되는 방법(dropout과 Batch gradient descent)을 소개하며, 거의 어떤 신경 구조의 정확성과 훈련 속도를 높이는 데 사용할 수 있다.\
本章介绍了两种最广泛使用的方法（dropout和Batch gradient descent），可用于提高几乎任何神经结构的准确性和训练速度
